In [1]:
pip install wfdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import subprocess
import sys

print("Installing compatible protobuf version...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "protobuf==3.20.0", "-q"])
print("Done!")

# Sekarang restart kernel/notebook
print("\nKERNEL RESTART REQUIRED!")
print("Please restart the kernel now (Jupyter will prompt you)")

Installing compatible protobuf version...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 5.0 MB/s eta 0:00:00
Done!

KERNEL RESTART REQUIRED!
Please restart the kernel now (Jupyter will prompt you)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.0 which is incompatible.
google-cloud-secret-manager 2.25.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
google-cloud-vision 3.11.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
google-cloud-monitoring 2.28.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.

In [3]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from scipy.signal import filtfilt, butter
import wfdb

2025-12-14 08:41:46.260797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765701706.431960      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765701706.483530      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# =====================================================
# 1. IMPROVED PREPROCESSING (Match GUI)
# =====================================================

def notch_filter_formula2(signal, f0=50, fs=128, r=0.98):
    """Notch filter 50Hz"""
    w0 = 2 * np.pi * f0 / fs
    c = np.cos(w0)
    b = np.array([1, -2*c, 1])
    a = np.array([1, -2*r*c, r*r])
    return filtfilt(b, a, signal)

def bandpass_filter(signal, low=0.5, high=50, fs=128, order=4):
    """Bandpass 0.5-50Hz (matches GUI training!)"""
    nyq = 0.5 * fs
    b, a = butter(order, [low/nyq, high/nyq], btype='band')
    return filtfilt(b, a, signal)

def normalize_robust(signal):
    """Robust normalization using percentiles 1-99"""
    q1, q99 = np.percentile(signal, [1, 99])
    if q99 - q1 == 0:
        return np.zeros_like(signal)
    signal_clipped = np.clip(signal, q1, q99)
    return (signal_clipped - q1) / (q99 - q1)


In [5]:
# =====================================================
# 2. LOAD SVDB WITH PVC SAMPLES (NEW!)
# =====================================================

def load_svdb (sve_symbols={"A", "a", "J", "S", "s"},
                       normal_symbols={"V", "v","N"},
                       window=512,
                       fs=128,
                       mask_width=48,
                       random_state=42):

    rng = np.random.default_rng(random_state)
    
    svdb_records = [
        '800','801','802','803','804','805','806','807','808','809','811','812','820','821','822','823',
        '824','825','826','827','828','829','840','841','842','844','846','847','848','849','850','851',
        '852','853','854','855','856','857','858','859','860','861','862','863','864','865','866','867',
        '868','869','870','871','872','873','874','875','876','877','878','879','880','881','882','883',
        '884','885','886','887','888','889','890','891','892','893','894'
    ]

    X_sve = []
    y_sve = []
    X_non_sve = []  
    y_non_sve = []

    for rec in svdb_records:
        print(f"Loading {rec}... ", end="", flush=True)
        try:
            sig, fields = wfdb.rdsamp(f"/kaggle/input/svdbfix/{rec}")
            ann = wfdb.rdann(f"/kaggle/input/svdbfix/{rec}", "atr")
            ecg = sig[:, 1]

            # Preprocessing (match GUI!)
            ecg = notch_filter_formula2(ecg, 50, fs, r=0.98)
            ecg = bandpass_filter(ecg, 0.5, 50, fs, order=4)
            ecg = normalize_robust(ecg)

            # Create mask for SVE only
            mask = np.zeros(len(ecg))
            events = list(zip(ann.sample, ann.symbol))

            # Mark ONLY SVE symbols
            for samp, sym in events:
                if sym in sve_symbols:
                    st = max(0, samp - mask_width)
                    ed = min(len(ecg), samp + mask_width)
                    mask[st:ed] = 1

            total = len(ecg) // window
            for w in range(total):
                s = w * window
                e = s + window
                win_syms = {sym for samp, sym in events if s <= samp < e}

                # Check what symbols are present
                has_sve = any(sym in sve_symbols for sym in win_syms)
                has_pvc = any(sym in normal_symbols for sym in win_syms)
                has_unknown = any(sym not in sve_symbols and sym not in normal_symbols and sym != '' for sym in win_syms)

                # Skip if has unknown symbols
                if has_unknown:
                    continue

                win_sig = ecg[s:e]
                win_mask = mask[s:e]

                # IMPORTANT: If has SVE, go to SVE class
                if has_sve:
                    X_sve.append(win_sig)
                    y_sve.append(win_mask)
                # Otherwise (PVC or Normal), go to non-SVE class
                else:
                    X_non_sve.append(win_sig)
                    y_non_sve.append(win_mask)  # All zeros for non-SVE

            print("✓")
        except Exception as e:
            print(f"Error: {e}")

    X_sve = np.array(X_sve).reshape(-1, window, 1)
    y_sve = np.array(y_sve).reshape(-1, window, 1)
    X_non_sve = np.array(X_non_sve).reshape(-1, window, 1)
    y_non_sve = np.array(y_non_sve).reshape(-1, window, 1)

    print(f"\nDataset before balancing:")
    print(f"SVE samples: {X_sve.shape[0]}")
    print(f"Non-SVE (PVC+Normal) samples: {X_non_sve.shape[0]}")

    # Balance: keep same amount of SVE and non-SVE
    target = min(X_sve.shape[0], X_non_sve.shape[0])

    if X_sve.shape[0] > target:
        idx = rng.choice(X_sve.shape[0], target, replace=False)
        X_sve = X_sve[idx]
        y_sve = y_sve[idx]

    if X_non_sve.shape[0] > target:
        idx = rng.choice(X_non_sve.shape[0], target, replace=False)
        X_non_sve = X_non_sve[idx]
        y_non_sve = y_non_sve[idx]

    # Combine
    X_all = np.concatenate([X_sve, X_non_sve], 0)
    y_all = np.concatenate([y_sve, y_non_sve], 0)

    print(f"\nDataset after balancing:")
    print(f"SVE: {X_sve.shape[0]}")
    print(f"Non-SVE: {X_non_sve.shape[0]}")
    print(f"Total: {X_all.shape}")

    # Shuffle
    idx = rng.permutation(X_all.shape[0])
    return X_all[idx], y_all[idx]

In [6]:
# =====================================================
# 3. LOSS FUNCTIONS & METRICS
# =====================================================

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    """Dice loss"""
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) +
                                           tf.reduce_sum(y_pred_f) + smooth)

def weighted_focal_tversky_loss(y_true, y_pred,
                                alpha=0.6, beta=0.4, gamma=2.5, smooth=1e-6):
    """Weighted focal tversky loss"""
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    tp = tf.reduce_sum(y_true_f * y_pred_f)
    fp = tf.reduce_sum((1 - y_true_f) * y_pred_f)
    fn = tf.reduce_sum(y_true_f * (1 - y_pred_f))
    tversky = (tp + smooth) / (tp + alpha*fn + beta*fp + smooth)
    return tf.pow((1 - tversky), gamma)

def norm_accuracy(y_true, y_pred, threshold=0.5, smooth=1e-6):
    """Normalized accuracy"""
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]) > threshold, tf.float32)
    diff = y_pred_f - y_true_f
    norm = tf.norm(diff, ord=2)
    denom = tf.sqrt(tf.cast(tf.size(y_true_f), tf.float32))
    return 1.0 - (norm / (denom + smooth))

In [7]:
def compare_preprocessing_multiple_records(record_list=['800', '801', '802'], 
                                          start_sample=0, 
                                          window_size=1024,
                                          fs=128):
    """
    Bandingkan preprocessing untuk beberapa record sekaligus
    
    Args:
        record_list: list nama record SVDB
        start_sample: sample awal
        window_size: ukuran window
        fs: sampling frequency
    """
    
    n_records = len(record_list)
    fig, axes = plt.subplots(n_records, 4, figsize=(20, 4*n_records))
    
    if n_records == 1:
        axes = axes.reshape(1, -1)
    
    fig.suptitle(f'Preprocessing Comparison - {n_records} Records\n'
                 f'Window: {window_size} samples ({window_size/fs:.1f}s)', 
                 fontsize=16, fontweight='bold', y=0.995)
    
    all_results = {}
    
    for idx, record_name in enumerate(record_list):
        print(f"\nProcessing record {record_name}...")
        
        try:
            # Load data
            sig, fields = wfdb.rdsamp(f"/kaggle/input/svdbfix/{record_name}")
            ecg_raw = sig[:, 1]
            
            # Extract window
            end_sample = min(start_sample + window_size, len(ecg_raw))
            ecg_segment = ecg_raw[start_sample:end_sample]
            
            # Process stages
            stage1_raw = ecg_segment.copy()
            stage2_notch = notch_filter_formula2(stage1_raw, f0=50, fs=fs, r=0.98)
            stage3_bandpass = bandpass_filter(stage2_notch, low=0.5, high=50, fs=fs, order=4)
            stage4_normalized = normalize_robust(stage3_bandpass)
            
            # Time axis
            time = np.arange(len(ecg_segment)) / fs
            
            # Plot all 4 stages
            stages = [
                (stage1_raw, 'Raw Signal', 'blue'),
                (stage2_notch, 'After Notch', 'green'),
                (stage3_bandpass, 'After Bandpass', 'orange'),
                (stage4_normalized, 'Normalized', 'red')
            ]
            
            for col, (data, title, color) in enumerate(stages):
                ax = axes[idx, col]
                ax.plot(time, data, color=color, linewidth=0.7, alpha=0.8)
                
                if idx == 0:
                    ax.set_title(title, fontweight='bold', fontsize=11)
                
                if col == 0:
                    ax.set_ylabel(f'Record {record_name}\n(mV)', fontsize=9, fontweight='bold')
                elif col == 3:
                    ax.set_ylabel(f'Record {record_name}\n(norm)', fontsize=9, fontweight='bold')
                
                if idx == n_records - 1:
                    ax.set_xlabel('Time (s)', fontsize=9)
                
                ax.grid(True, alpha=0.3, linewidth=0.5)
                ax.set_xlim([time[0], time[-1]])
                
                # Add stats box
                stats = f'μ={np.mean(data):.2f}\nσ={np.std(data):.2f}'
                ax.text(0.02, 0.98, stats, transform=ax.transAxes,
                       fontsize=8, verticalalignment='top',
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
            
            # Store results
            all_results[record_name] = {
                'raw': stage1_raw,
                'notch': stage2_notch,
                'bandpass': stage3_bandpass,
                'normalized': stage4_normalized
            }
            
            print(f"  ✓ {record_name} processed")
            
        except Exception as e:
            print(f"  ❌ Error processing {record_name}: {e}")
            continue
    
    plt.tight_layout()
    plt.savefig(f'preprocessing_comparison_{len(record_list)}_records.png', 
                dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n✓ Saved: preprocessing_comparison_{len(record_list)}_records.png")
    
    return all_results


def show_frequency_spectrum(record_name='800', start_sample=0, window_size=2048, fs=128):
    """
    Tampilkan spektrum frekuensi sebelum dan sesudah filtering
    """
    
    print(f"Analyzing frequency spectrum for record {record_name}...")
    
    # Load data
    sig, fields = wfdb.rdsamp(f"/kaggle/input/svdbfix/{record_name}")
    ecg_raw = sig[:, 1]
    
    # Extract window
    end_sample = min(start_sample + window_size, len(ecg_raw))
    ecg_segment = ecg_raw[start_sample:end_sample]
    
    # Process
    stage1_raw = ecg_segment.copy()
    stage2_notch = notch_filter_formula2(stage1_raw, f0=50, fs=fs, r=0.98)
    stage3_bandpass = bandpass_filter(stage2_notch, low=0.5, high=50, fs=fs, order=4)
    stage4_normalized = normalize_robust(stage3_bandpass)
    
    # Compute FFT
    def compute_fft(signal, fs):
        n = len(signal)
        fft_vals = np.fft.rfft(signal)
        fft_freq = np.fft.rfftfreq(n, 1/fs)
        fft_power = np.abs(fft_vals) ** 2
        return fft_freq, fft_power
    
    freq1, power1 = compute_fft(stage1_raw, fs)
    freq2, power2 = compute_fft(stage2_notch, fs)
    freq3, power3 = compute_fft(stage3_bandpass, fs)
    
    # Plot
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Time domain - before
    axes[0, 0].plot(np.arange(len(stage1_raw))/fs, stage1_raw, 'b-', linewidth=0.7, alpha=0.8)
    axes[0, 0].set_title('Time Domain - Raw Signal', fontweight='bold', fontsize=12)
    axes[0, 0].set_xlabel('Time (s)')
    axes[0, 0].set_ylabel('Amplitude (mV)')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Frequency domain - before
    axes[0, 1].semilogy(freq1, power1, 'b-', linewidth=1, alpha=0.8)
    axes[0, 1].axvline(50, color='red', linestyle='--', linewidth=2, label='50Hz (Power Line)')
    axes[0, 1].set_title('Frequency Domain - Raw Signal', fontweight='bold', fontsize=12)
    axes[0, 1].set_xlabel('Frequency (Hz)')
    axes[0, 1].set_ylabel('Power (log scale)')
    axes[0, 1].set_xlim([0, fs/2])
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Time domain - after filtering
    axes[1, 0].plot(np.arange(len(stage3_bandpass))/fs, stage3_bandpass, 'orange', linewidth=0.7, alpha=0.8)
    axes[1, 0].set_title('Time Domain - After Filtering', fontweight='bold', fontsize=12)
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Amplitude (mV)')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Frequency domain - after filtering
    axes[1, 1].semilogy(freq3, power3, 'orange', linewidth=1, alpha=0.8)
    axes[1, 1].axvline(0.5, color='green', linestyle='--', linewidth=1.5, label='0.5Hz (Low cutoff)')
    axes[1, 1].axvline(50, color='red', linestyle='--', linewidth=1.5, label='50Hz (High cutoff)')
    axes[1, 1].set_title('Frequency Domain - After Filtering', fontweight='bold', fontsize=12)
    axes[1, 1].set_xlabel('Frequency (Hz)')
    axes[1, 1].set_ylabel('Power (log scale)')
    axes[1, 1].set_xlim([0, fs/2])
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.suptitle(f'Frequency Analysis - Record {record_name}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f'frequency_analysis_{record_name}.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✓ Saved: frequency_analysis_{record_name}.png")

In [8]:
# =====================================================
# USAGE EXAMPLES
# =====================================================

print("\n" + "="*70)
print("PREPROCESSING VISUALIZATION TOOLS")
print("="*70)

print("\n1. Compare multiple records:")
print("   results = compare_preprocessing_multiple_records(['800', '801', '802'])")

print("\n2. Analyze frequency spectrum:")
print("   show_frequency_spectrum('800', start_sample=0, window_size=2048)")

print("\n3. Custom comparison:")
print("   results = compare_preprocessing_multiple_records(")
print("       record_list=['800', '825', '850'],")
print("       start_sample=1000,")
print("       window_size=512)")

print("\n" + "="*70)


PREPROCESSING VISUALIZATION TOOLS

1. Compare multiple records:
   results = compare_preprocessing_multiple_records(['800', '801', '802'])

2. Analyze frequency spectrum:
   show_frequency_spectrum('800', start_sample=0, window_size=2048)

3. Custom comparison:
   results = compare_preprocessing_multiple_records(
       record_list=['800', '825', '850'],
       start_sample=1000,
       window_size=512)



In [9]:
# =====================================================
# 4. BUILD UNET 1D MODEL
# =====================================================

from tensorflow import keras
from tensorflow.keras import layers, models

def build_unet_1d(input_length=512, dr=0.15):
    """UNet 1D architecture"""
    inputs = layers.Input((input_length, 1))
    
    # Encoder 1
    c1 = layers.Conv1D(32, 3, padding='same', activation='relu')(inputs)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Dropout(dr)(c1)
    c1 = layers.Conv1D(32, 3, padding='same', activation='relu')(c1)
    c1 = layers.BatchNormalization()(c1)
    p1 = layers.MaxPooling1D(2)(c1)
    
    # Encoder 2
    c2 = layers.Conv1D(64, 3, padding='same', activation='relu')(p1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Dropout(dr)(c2)
    c2 = layers.Conv1D(64, 3, padding='same', activation='relu')(c2)
    c2 = layers.BatchNormalization()(c2)
    p2 = layers.MaxPooling1D(2)(c2)
    
    # Encoder 3
    c3 = layers.Conv1D(128, 3, padding='same', activation='relu')(p2)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Dropout(dr)(c3)
    c3 = layers.Conv1D(128, 3, padding='same', activation='relu')(c3)
    c3 = layers.BatchNormalization()(c3)
    p3 = layers.MaxPooling1D(2)(c3)
    
    # Encoder 4
    c4 = layers.Conv1D(256, 3, padding='same', activation='relu')(p3)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Dropout(dr)(c4)
    c4 = layers.Conv1D(256, 3, padding='same', activation='relu')(c4)
    c4 = layers.BatchNormalization()(c4)
    p4 = layers.MaxPooling1D(2)(c4)
    
    # Bottleneck
    bn = layers.Conv1D(512, 3, padding='same', activation='relu')(p4)
    bn = layers.BatchNormalization()(bn)
    bn = layers.Dropout(dr)(bn)
    bn = layers.Conv1D(512, 3, padding='same', activation='relu')(bn)
    bn = layers.BatchNormalization()(bn)
    
    # Decoder 1
    u4 = layers.UpSampling1D(2)(bn)
    u4 = layers.Concatenate()([u4, c4])
    d4 = layers.Conv1D(256, 3, padding='same', activation='relu')(u4)
    d4 = layers.BatchNormalization()(d4)
    d4 = layers.Dropout(dr)(d4)
    d4 = layers.Conv1D(256, 3, padding='same', activation='relu')(d4)
    d4 = layers.BatchNormalization()(d4)
    
    # Decoder 2
    u3 = layers.UpSampling1D(2)(d4)
    u3 = layers.Concatenate()([u3, c3])
    d3 = layers.Conv1D(128, 3, padding='same', activation='relu')(u3)
    d3 = layers.BatchNormalization()(d3)
    d3 = layers.Dropout(dr)(d3)
    d3 = layers.Conv1D(128, 3, padding='same', activation='relu')(d3)
    d3 = layers.BatchNormalization()(d3)
    
    # Decoder 3
    u2 = layers.UpSampling1D(2)(d3)
    u2 = layers.Concatenate()([u2, c2])
    d2 = layers.Conv1D(64, 3, padding='same', activation='relu')(u2)
    d2 = layers.BatchNormalization()(d2)
    d2 = layers.Dropout(dr)(d2)
    d2 = layers.Conv1D(64, 3, padding='same', activation='relu')(d2)
    d2 = layers.BatchNormalization()(d2)
    
    # Decoder 4
    u1 = layers.UpSampling1D(2)(d2)
    u1 = layers.Concatenate()([u1, c1])
    d1 = layers.Conv1D(32, 3, padding='same', activation='relu')(u1)
    d1 = layers.BatchNormalization()(d1)
    d1 = layers.Dropout(dr)(d1)
    d1 = layers.Conv1D(32, 3, padding='same', activation='relu')(d1)
    d1 = layers.BatchNormalization()(d1)
    
    # Output
    outputs = layers.Conv1D(1, 1, activation='sigmoid')(d1)
    
    return models.Model(inputs, outputs)

In [10]:
# =====================================================
# 5. TRAINING (50-FOLD CV)
# =====================================================

print("="*70)
print("LOADING SVDB DATASET WITH PVC SAMPLES")
print("="*70)
X, y = load_svdb()
print(f"\nDataset shape: {X.shape}, {y.shape}")

print("\n" + "="*70)
print("STARTING 50-FOLD CROSS VALIDATION WITH PVC TRAINING")
print("="*70)

kfold = KFold(n_splits=50, shuffle=True, random_state=42)
all_loss = []
all_dice = []
all_norm = []
all_prec = []
all_rec = []
all_f1 = []

fold = 1
for train_idx, val_idx in kfold.split(X):
    print(f"\nFOLD {fold}/50 | Train {len(train_idx)} | Val {len(val_idx)}")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Build model
    model = build_unet_1d(512, dr=0.15)

    # Compile
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
        loss='binary_crossentropy',
        metrics=[
            dice_coefficient,
            norm_accuracy,
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=32,
        verbose=1,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='val_recall',
                patience=12,
                restore_best_weights=True,
                mode='max'),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-6)
        ]
    )

    # Evaluate
    results = model.evaluate(X_val, y_val, verbose=0)
    loss, dice, norm_acc, prec, rec = results
    f1 = 2 * (prec * rec) / (prec + rec + 1e-6)

    print(f"  Results: Dice={dice*100:.1f}% | NormAcc={norm_acc*100:.1f}% | "
          f"P={prec*100:.1f}% | R={rec*100:.1f}% | F1={f1*100:.1f}%")

    all_loss.append(loss)
    all_dice.append(dice)
    all_norm.append(norm_acc)
    all_prec.append(prec)
    all_rec.append(rec)
    all_f1.append(f1)

    # Save model
    model.save(f"unet_fold_{fold}_with_pvc.h5")
    print(f"  Saved: unet_fold_{fold}_with_pvc.h5")
    
    fold += 1

LOADING SVDB DATASET WITH PVC SAMPLES
Loading 800... ✓
Loading 801... ✓
Loading 802... ✓
Loading 803... ✓
Loading 804... ✓
Loading 805... ✓
Loading 806... ✓
Loading 807... ✓
Loading 808... ✓
Loading 809... ✓
Loading 811... ✓
Loading 812... ✓
Loading 820... ✓
Loading 821... ✓
Loading 822... ✓
Loading 823... ✓
Loading 824... ✓
Loading 825... ✓
Loading 826... ✓
Loading 827... ✓
Loading 828... ✓
Loading 829... ✓
Loading 840... ✓
Loading 841... ✓
Loading 842... ✓
Loading 844... ✓
Loading 846... ✓
Loading 847... ✓
Loading 848... ✓
Loading 849... ✓
Loading 850... ✓
Loading 851... ✓
Loading 852... ✓
Loading 853... ✓
Loading 854... ✓
Loading 855... ✓
Loading 856... ✓
Loading 857... ✓
Loading 858... ✓
Loading 859... ✓
Loading 860... ✓
Loading 861... ✓
Loading 862... ✓
Loading 863... ✓
Loading 864... ✓
Loading 865... ✓
Loading 866... ✓
Loading 867... ✓
Loading 868... ✓
Loading 869... ✓
Loading 870... ✓
Loading 871... ✓
Loading 872... ✓
Loading 873... ✓
Loading 874... ✓
Loading 875... ✓
Loading 87

I0000 00:00:1765701732.636659      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Epoch 1/100


I0000 00:00:1765701752.039770     117 service.cc:148] XLA service 0x7f8fb8002d60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765701752.040397     117 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1765701753.859192     117 cuda_dnn.cc:529] Loaded cuDNN version 90300


  7/395 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - dice_coefficient: 0.1992 - loss: 0.8473 - norm_accuracy: 0.3121 - precision: 0.1270 - recall: 0.4744  

I0000 00:00:1765701768.187002     117 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


KeyboardInterrupt: 

In [ ]:
# =====================================================
# 6. RESULTS SUMMARY
# =====================================================

print("\n" + "="*70)
print("TRAINING COMPLETE - FINAL RESULTS")
print("="*70)
print(f"\nAverage Dice Coefficient: {np.mean(all_dice)*100:.2f}% ± {np.std(all_dice)*100:.2f}%")
print(f"Average Norm Accuracy:    {np.mean(all_norm)*100:.2f}% ± {np.std(all_norm)*100:.2f}%")
print(f"Average Precision:        {np.mean(all_prec)*100:.2f}% ± {np.std(all_prec)*100:.2f}%")
print(f"Average Recall:           {np.mean(all_rec)*100:.2f}% ± {np.std(all_rec)*100:.2f}%")
print(f"Average F1 Score:         {np.mean(all_f1)*100:.2f}% ± {np.std(all_f1)*100:.2f}%")

# Select best model
best_fold = np.argmax(all_f1) + 1
print(f"\n✅ Best model: unet_fold_{best_fold}_with_pvc.h5")
print(f"   F1 Score: {all_f1[best_fold-1]*100:.2f}%")

print("\n✅ TRAINING DONE!")
print("Next: Load best model into GUI and test")

In [ ]:
# =====================================================
# ENSEMBLE - SIMPLE VERSION
# =====================================================

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

print("\nLoading 50 models...")

models_ensemble = []
fold_dice_list = []

for fold_no in range(1, len(all_dice) + 1):
    model_path = f"unet_fold_{fold_no}_with_pvc.h5"
    
    try:
        model = load_model(model_path, compile=False)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            loss=lambda y_true, y_pred: dice_bce_hybrid_loss(y_true, y_pred, alpha=0.5),
            metrics=[dice_coefficient, norm_accuracy]
        )
        models_ensemble.append(model)
        fold_dice_list.append(float(all_dice[fold_no - 1]))
        
        if fold_no % 10 == 0:
            print(f"  Fold {fold_no}: OK")
    except:
        pass

print(f"Loaded: {len(models_ensemble)}/50\n")

# =====================================================
# ENSEMBLE FUNCTIONS
# =====================================================

def ensemble_predict_average(X):
    preds = [model.predict(X, verbose=0) for model in models_ensemble]
    return np.mean(preds, axis=0)

def ensemble_predict_weighted(X):
    preds = [model.predict(X, verbose=0) for model in models_ensemble]
    weights = np.array(fold_dice_list) / np.sum(fold_dice_list)
    result = np.zeros_like(preds[0])
    for pred, w in zip(preds, weights):
        result += pred * w
    return result

def ensemble_predict_median(X):
    preds = [model.predict(X, verbose=0) for model in models_ensemble]
    return np.median(preds, axis=0)

# =====================================================
# TEST ENSEMBLE
# =====================================================

print("Testing ensemble...\n")

# Test data
test_size = min(200, X_val.shape[0])
X_test = X_val[:test_size]
y_test = tf.cast(y_val[:test_size], tf.float32)

# Average
pred_avg = tf.cast(ensemble_predict_average(X_test), tf.float32)
dice_avg = dice_coefficient(y_test, pred_avg).numpy()

# Weighted
pred_weighted = tf.cast(ensemble_predict_weighted(X_test), tf.float32)
dice_weighted = dice_coefficient(y_test, pred_weighted).numpy()

# Median
pred_median = tf.cast(ensemble_predict_median(X_test), tf.float32)
dice_median = dice_coefficient(y_test, pred_median).numpy()

# Results
avg_ind = np.mean(all_dice)

print(f"Individual Avg: {avg_ind*100:.2f}%")
print(f"Average Ensemble: {dice_avg*100:.2f}%")
print(f"Weighted Ensemble: {dice_weighted*100:.2f}%")
print(f"Median Ensemble: {dice_median*100:.2f}%")

# Best method
best = max(dice_avg, dice_weighted, dice_median)
if best == dice_weighted:
    best_method = 'weighted'
elif best == dice_avg:
    best_method = 'average'
else:
    best_method = 'median'

print(f"\nBest: {best_method.upper()} ({best*100:.2f}%)")

# =====================================================
# INFERENCE FUNCTION
# =====================================================

def predict_with_ensemble(ecg_512):
    """Predict dengan best method"""
    if ecg_512.ndim == 1:
        ecg_512 = ecg_512.reshape(-1, 1)
    X = ecg_512.reshape(1, 512, 1)
    
    if best_method == 'weighted':
        return ensemble_predict_weighted(X).squeeze()
    elif best_method == 'median':
        return ensemble_predict_median(X).squeeze()
    else:
        return ensemble_predict_average(X).squeeze()

print("\nOK! Use: predict_with_ensemble(X_test[0])")

In [ ]:
# =====================================================
# CREATE FINAL MODEL
# =====================================================

print("\n" + "="*60)
print("CREATING FINAL MODEL")
print("="*60)

# Get ensemble predictions
print("\nGenerating pseudo-labels dari ensemble...")
y_ensemble = np.mean([model.predict(X, verbose=0) for model in models_ensemble], axis=0)
print(f"✓ Shape: {y_ensemble.shape}")

# Build model
print("\nBuilding UNet model...")
final_model = build_unet_1d(512)

final_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss='binary_crossentropy',
    metrics=[dice_coefficient, norm_accuracy]
)

# Train
print("\nTraining...")
history = final_model.fit(
    X, y_ensemble,
    epochs=100,
    batch_size=128,
    shuffle=True,
    verbose=1
)

# Save
final_model.save("unet_svdb_FINAL.h5")
print("\n✓ Saved: unet_svdb_FINAL.h5")

# Test
print("\n" + "="*60)
print("TEST")
print("="*60)

test_size = min(200, X_val.shape[0])
X_test = X_val[:test_size]
y_test = tf.cast(y_val[:test_size], tf.float32)

pred = tf.cast(final_model.predict(X_test, verbose=0), tf.float32)
dice = dice_coefficient(y_test, pred).numpy()

print(f"\nFinal Model Dice: {dice*100:.2f}%")
print(f"✓ READY FOR INFERENCE!")

# Function
def predict_final_model(ecg_512):
    if ecg_512.ndim == 1:
        ecg_512 = ecg_512.reshape(-1, 1)
    X = ecg_512.reshape(1, 512, 1)
    return final_model.predict(X, verbose=0).squeeze()

print("\nUsage: pred = predict_final_model(X_test[0])")

In [ ]:
# =====================================================
# EVALUATE FINAL MODEL - Run AFTER training complete
# =====================================================

import tensorflow as tf
import numpy as np

print("\n" + "="*60)
print("LOADING TRAINED MODEL")
print("="*60)

# Load the model that was trained
final_model = tf.keras.models.load_model("unet_svdb_FINAL.h5")
print("✓ Model loaded: unet_svdb_FINAL.h5")

# =====================================================
# PREPARE TEST DATA
# =====================================================

print("\n" + "="*60)
print("PREPARING TEST DATA")
print("="*60)

test_size = min(200, X_val.shape[0])
X_test = X_val[:test_size]
y_test = tf.cast(y_val[:test_size], tf.float32)

print(f"✓ Test set size: {test_size} samples")
print(f"✓ X_test shape: {X_test.shape}")
print(f"✓ y_test shape: {y_test.shape}")

# =====================================================
# GET PREDICTIONS
# =====================================================

print("\n" + "="*60)
print("RUNNING INFERENCE")
print("="*60)

# Get probability predictions
pred_probs = final_model.predict(X_test, verbose=0)
pred_probs = tf.cast(pred_probs, tf.float32)
print("✓ Got probability predictions")

# Convert to binary (threshold 0.5)
pred_binary = tf.cast(pred_probs > 0.5, tf.float32)
print("✓ Converted to binary predictions (threshold=0.5)")

# =====================================================
# FLATTEN FOR METRICS
# =====================================================

y_test_flat = tf.reshape(y_test, [-1])
pred_binary_flat = tf.reshape(pred_binary, [-1])
pred_probs_flat = tf.reshape(pred_probs, [-1])

print(f"\n✓ Flattened for metric calculation")
print(f"  Total predictions: {len(pred_binary_flat):,} samples")

# =====================================================
# CALCULATE CONFUSION MATRIX
# =====================================================

print("\n" + "="*60)
print("CALCULATING METRICS")
print("="*60)

tp = tf.reduce_sum(y_test_flat * pred_binary_flat)
fp = tf.reduce_sum((1 - y_test_flat) * pred_binary_flat)
fn = tf.reduce_sum(y_test_flat * (1 - pred_binary_flat))
tn = tf.reduce_sum((1 - y_test_flat) * (1 - pred_binary_flat))

print(f"\nConfusion Matrix:")
print(f"  TP (True Positive):   {int(tp.numpy()):,}")
print(f"  FP (False Positive):  {int(fp.numpy()):,}")
print(f"  TN (True Negative):   {int(tn.numpy()):,}")
print(f"  FN (False Negative):  {int(fn.numpy()):,}")

# =====================================================
# CALCULATE METRICS
# =====================================================

# Precision
precision = tp / (tp + fp + 1e-7)

# Recall (Sensitivity)
recall = tp / (tp + fn + 1e-7)

# F1 Score
f1 = 2 * (precision * recall) / (precision + recall + 1e-7)

# Accuracy
accuracy = (tp + tn) / (tp + fp + fn + tn + 1e-7)

# Specificity
specificity = tn / (tn + fp + 1e-7)

# Dice Coefficient
dice = dice_coefficient(y_test, pred_probs).numpy()

# Norm Accuracy
norm_acc = norm_accuracy(y_test, pred_probs).numpy()

# =====================================================
# PRINT ALL RESULTS
# =====================================================

print("\n" + "="*60)
print("FINAL MODEL EVALUATION RESULTS")
print("="*60)

print(f"\nMetrics on validation set ({test_size} samples):")
print(f"{'='*60}")
print(f"Precision:        {precision.numpy()*100:6.2f}%")
print(f"Recall:           {recall.numpy()*100:6.2f}%")
print(f"Specificity:      {specificity.numpy()*100:6.2f}%")
print(f"F1 Score:         {f1.numpy()*100:6.2f}%")
print(f"Accuracy:         {accuracy.numpy()*100:6.2f}%")
print(f"Dice Coefficient: {dice*100:6.2f}%")
print(f"Norm Accuracy:    {norm_acc*100:6.2f}%")
print(f"{'='*60}")

# =====================================================
# CONFUSION MATRIX DETAILS
# =====================================================

print(f"\nConfusion Matrix Details:")
print(f"{'='*60}")
print(f"True Positives:   {int(tp.numpy()):,}")
print(f"False Positives:  {int(fp.numpy()):,}")
print(f"True Negatives:   {int(tn.numpy()):,}")
print(f"False Negatives:  {int(fn.numpy()):,}")
print(f"{'='*60}")

# =====================================================
# QUALITY ASSESSMENT
# =====================================================

print(f"\n" + "="*60)
print("MODEL QUALITY ASSESSMENT")
print("="*60)

score_list = []

if recall.numpy() > 0.85:
    print(f"  ✅ Recall EXCELLENT ({recall.numpy()*100:.2f}%)")
    score_list.append("excellent")
elif recall.numpy() > 0.80:
    print(f"  ✅ Recall GOOD ({recall.numpy()*100:.2f}%)")
    score_list.append("good")
else:
    print(f"  ⚠️  Recall OK ({recall.numpy()*100:.2f}%)")
    score_list.append("ok")

if f1.numpy() > 0.83:
    print(f"  ✅ F1 EXCELLENT ({f1.numpy()*100:.2f}%)")
    score_list.append("excellent")
elif f1.numpy() > 0.80:
    print(f"  ✅ F1 GOOD ({f1.numpy()*100:.2f}%)")
    score_list.append("good")
else:
    print(f"  ⚠️  F1 OK ({f1.numpy()*100:.2f}%)")
    score_list.append("ok")

if precision.numpy() > 0.80:
    print(f"  ✅ Precision GOOD ({precision.numpy()*100:.2f}%)")
    score_list.append("good")
elif precision.numpy() > 0.75:
    print(f"  ⚠️  Precision OK ({precision.numpy()*100:.2f}%)")
    score_list.append("ok")
else:
    print(f"  ❌ Precision LOW ({precision.numpy()*100:.2f}%)")
    score_list.append("low")

# Final recommendation
print(f"\n" + "="*60)
if "excellent" in score_list and recall.numpy() > 0.85:
    print("🎉 READY FOR PRODUCTION!")
    print("✅ Model performs excellently")
    print("✅ Can be deployed to GUI")
elif "good" in score_list and recall.numpy() > 0.80:
    print("✅ READY FOR DEPLOYMENT")
    print("✅ Model performs well")
    print("✅ Acceptable for production")
else:
    print("⚠️  NEEDS IMPROVEMENT")
    print("⚠️  Consider retraining")
    print("⚠️  Adjust hyperparameters")

print("="*60)

# =====================================================
# SAVE RESULTS TO FILE
# =====================================================

print("\n" + "="*60)
print("SAVING RESULTS")
print("="*60)

# Create results dictionary
results = {
    'precision': float(precision.numpy()),
    'recall': float(recall.numpy()),
    'specificity': float(specificity.numpy()),
    'f1': float(f1.numpy()),
    'accuracy': float(accuracy.numpy()),
    'dice': float(dice),
    'norm_accuracy': float(norm_acc),
    'tp': int(tp.numpy()),
    'fp': int(fp.numpy()),
    'tn': int(tn.numpy()),
    'fn': int(fn.numpy()),
    'test_samples': test_size
}

# Save to JSON
import json
with open('final_model_evaluation.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✓ Results saved to: final_model_evaluation.json")

# =====================================================
# INFERENCE FUNCTION
# =====================================================

print("\n" + "="*60)
print("READY FOR INFERENCE")
print("="*60)

def predict_final_model(ecg_512):
    """
    Predict SVE mask for 512-sample ECG window
    
    Args:
        ecg_512: numpy array of shape (512,) or (512, 1)
    
    Returns:
        Probability mask (512,) with values 0-1
    """
    if ecg_512.ndim == 1:
        ecg_512 = ecg_512.reshape(-1, 1)
    X = ecg_512.reshape(1, 512, 1)
    return final_model.predict(X, verbose=0).squeeze()

print("\nUsage examples:")
print("  # Get probability prediction")
print("  pred = predict_final_model(X_test[0])")
print("\n  # Get binary mask")
print("  mask = (pred > 0.5).astype(int)")
print("\n  # Get SVE burden")
print("  burden = np.mean(mask) * 100")
print("\n✓ Ready to use!")